# Automating Crypto Website API Pull

This project demonstrates how to automate the extraction of cryptocurrency data from a website API using Python. It covers setting up the API connection, sending requests, and processing the returned JSON data. 

The project includes code to handle potential errors, ensure data integrity, and efficiently store the retrieved information for further analysis or real-time updates. This automation is ideal for tasks like tracking price changes, portfolio management, or feeding data into a larger trading system. 

The Python libraries used include requests for API calls and pandas for data manipulation.

In [ ]:
# Getting the data from https://coinmarketcap.com/api/ as per documentation

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'30',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '5a18d06a-f78b-4b2b-8d87-50c2485a1c91',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#NOTE:
# When running this code I encounter an error that limitates the data rate
# To fix it I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10"
# into the Anaconda Prompt and use the provided local host URL to keep working
# After that the code was able to run as expected

In [33]:
type(data)

dict

In [35]:
import pandas as pd

# Allowing the columns from this dataset to be displayed in full

pd.set_option('display.max_columns', None)

In [37]:
# Normalizing the data for better overview

df = pd.json_normalize(data['data'])

# Adding a timestamp to the dataset to keep track of the data collection

df['timestamp'] = pd.to_datetime('now')
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,11633,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.973998e+07,1.973998e+07,False,NaN,1,NaN,NaN,NaN,2024-08-15T07:40:00.000Z,57972.579648,3.253490e+10,12.5358,-0.368119,-4.727788,1.075420,-7.988078,-12.494066,-12.350869,1.144377e+12,55.6517,1.217424e+12,NaN,2024-08-15T07:40:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 08:42:38.651678
1,1027,Ethereum,ETH,ethereum,9210,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.202770e+08,1.202770e+08,True,NaN,2,NaN,NaN,NaN,2024-08-15T07:39:00.000Z,2611.776025,1.592308e+10,5.3451,-0.501807,-4.127623,8.014539,-23.058498,-26.712605,-13.673078,3.141367e+11,15.2899,3.141367e+11,NaN,2024-08-15T07:39:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 08:42:38.651678
2,825,Tether USDt,USDT,tether,94168,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, avalanch...",NaN,1.165322e+11,1.180603e+11,True,NaN,3,NaN,NaN,NaN,2024-08-15T07:39:00.000Z,1.000176,5.437938e+10,10.2363,0.017222,-0.033487,-0.023522,-0.033945,0.081678,-0.041556,1.165527e+11,5.6680,1.180811e+11,NaN,2024-08-15T07:39:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2024-08-15 08:42:38.651678
3,1839,BNB,BNB,bnb,2208,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.459363e+08,1.459363e+08,False,NaN,4,NaN,NaN,NaN,2024-08-15T07:39:00.000Z,515.916751,1.621371e+09,-5.0565,-0.371449,-2.047567,6.064996,-8.975936,-14.875279,-10.608008,7.529100e+10,3.6685,7.529100e+10,NaN,2024-08-15T07:39:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 08:42:38.651678
4,5426,Solana,SOL,solana,716,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.668354e+08,5.822595e+08,True,NaN,5,NaN,NaN,NaN,2024-08-15T07:39:00.000Z,141.765633,2.751697e+09,29.7328,0.024867,-2.391038,-8.510384,-8.649242,-1.984186,-16.033400,6.618121e+10,3.2212,8.254439e+10,NaN,2024-08-15T07:39:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 08:42:38.651678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6210,The Sandbox,SAND,the-sandbox,572,2020-08-05T00:00:00.000Z,"[collectibles-nfts, content-creation, gaming, ...",NaN,2.284232e+09,3.000000e+09,False,NaN,96,NaN,NaN,NaN,2024-08-15T07:40:00.000Z,0.254786,4.582054e+07,17.8305,-0.236351,-2.531881,2.323087,-22.637337,-33.738734,-42.687852,5.819910e+08,0.0283,7.643589e+08,NaN,2024-08-15T07:40:00.000Z,1027.0,Ethereum,ETH,ethereum,0x3845badAde8e6dFF049820680d1F14bD3903a5d0,2024-08-15 08:42:38.651678
96,7334,Conflux,CFX,conflux-network,172,2020-11-11T00:00:00.000Z,"[mineable, pow, platform, collectibles-nfts, d...",NaN,4.332154e+09,5.032154e+09,True,NaN,97,4.332152e+09,5.803262e+08,NaN,2024-08-15T07:39:00.000Z,0.133958,4.084236e+07,49.2668,-0.117851,-3.479544,9.160980,-20.094176,-27.563065,-39.622190,5.803266e+08,0.0282,6.740972e+08,NaN,2024-08-15T07:39:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 08:42:38.651678
97,30171,Ethena,ENA,ethena,178,2024-03-29T04:42:08.000Z,"[defi, binance-labs-portfolio, okx-ventures-po...",NaN,1.806250e+09,1.500000e+10,False,NaN,98,1.479139e+09,4.653969e+08,NaN,2024-08-15T07:39:00.000Z,0.314640,4.569885e+07,-8.6862,-0.288980,-5.994366,12.992832,-31.199054,-57.399586,-55.650559,5.683192e+08,0.0276,4.7

In [8]:
# Automating the process through a function that will run over and over again
# and append the new data to the data it already has
# NOTE: This is the final and debugged code, after running into a few issues. See cell below for
# error log and debugging techniques.

import os
from time import sleep
import json
import pandas as pd
from requests import Session, ConnectionError, Timeout, TooManyRedirects

def api_runner():
    global df

    # Initialize an empty DataFrame if df doesn't exist
    if 'df' not in globals():
        df = pd.DataFrame()

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start': '1',
        'limit': '100',
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '5a18d06a-f78b-4b2b-8d87-50c2485a1c91',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        print("API request successful.")
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(f"API request failed: {e}")
        return  # Exit the function if an error occurs

    # Check if data is received correctly
    if not data or 'data' not in data:
        print("No data received from the API.")
        return

    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    print(f"Data normalized. Number of records fetched: {len(df2)}")

    # Concatenate new data with existing DataFrame
    df = pd.concat([df, df2], ignore_index=True)
    print(f"Data concatenated. Total records in DataFrame: {len(df)}")

    # Correct CSV file path
    csv_file_path = r"C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI.csv"

    # Write to CSV: create new file or append to existing file
    if not os.path.isfile(csv_file_path):
        df.to_csv(csv_file_path, index=False, mode='w', header=True)
        print(f"CSV file created at {csv_file_path}")
    else:
        df.to_csv(csv_file_path, index=False, mode='a', header=False)
        print(f"Data appended to existing CSV file at {csv_file_path}")

# Script to run the API pull multiple times
for i in range(333):
    print(f"Starting API Runner iteration {i+1}")
    api_runner()
    print('API Runner completed successfully')
    sleep(60)  # Sleep for 1 minute

Starting API Runner iteration 1
API request successful.
Data normalized. Number of records fetched: 100
Data concatenated. Total records in DataFrame: 103
CSV file created at C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI.csv
API Runner completed successfully
Starting API Runner iteration 2
API request successful.
Data normalized. Number of records fetched: 100
Data concatenated. Total records in DataFrame: 203
Data appended to existing CSV file at C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI.csv
API Runner completed successfully
Starting API Runner iteration 3
API request successful.
Data normalized. Number of records fetched: 100
Data concatenated. Total records in DataFrame: 303
Data appended to existing CSV file at C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI.csv
API Runner completed successfully


KeyboardInterrupt: 

In [6]:
# I encounter an issue trying to create a csv file to append the data when running the previous script
# To troubleshoot I created a test file creation separately to check if the issue persists and it did not
# Therefore it confirms that the csv creation logic is correct and the issue lies in the integration
# with the API data and the circumstance of when the CSV is being written. To fix that I have to debug the 
# the API script and update. The debugging steps I'm going to take is to add debugging statements
# to track where the process might be failing or stopping. And also add a data validation.
# For a more detail on the troubleshooting I created a documentation file included in my Python Repository on Github.

import os
import pandas as pd

# Dummy DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

# Correct CSV file path
csv_file_path = r"C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI_test.csv"

# Write to CSV: create new file or append to existing file
if not os.path.isfile(csv_file_path):
    df.to_csv(csv_file_path, index=False, mode='w', header=True)
    print(f"CSV file created at {csv_file_path}")
else:
    df.to_csv(csv_file_path, index=False, mode='a', header=False)
    print(f"Data appended to existing CSV file at {csv_file_path}")


CSV file created at C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI_test.csv


In [33]:
# Reading the csv we just created after a few runs to make sure it's working
df = pd.read_csv(r"C:\Users\gusta\Desktop\Dri - Projects\CryptoAPI.csv")
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1.00000,Bitcoin,BTC,bitcoin,11635.00000,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",21000000.00000,19739978.00000,19739978.00000,...,55.77930,1223809198245.47998,NaN,2024-08-15T08:34:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
1,1027.00000,Ethereum,ETH,ethereum,9211.00000,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,120277040.25230,120277040.25230,...,15.24350,314355730647.87000,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
2,825.00000,Tether USDt,USDT,tether,94195.00000,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'ava...",NaN,116532189829.29184,118060317087.51749,...,5.64760,118056657100.92000,NaN,2024-08-15T08:33:00.000Z,1027.00000,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2024-08-15 09:35:58.183530
3,1839.00000,BNB,BNB,bnb,2209.00000,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,145936326.94494,145936326.94494,...,3.66480,75598497131.34000,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
4,5426.00000,Solana,SOL,solana,716.00000,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,466835260.78769,582259423.78854,...,3.22420,82929110850.32001,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,6210.00000,The Sandbox,SAND,the-sandbox,573.00000,2020-08-05T00:00:00.000Z,"['collectibles-nfts', 'content-creation', 'gam...",NaN,2284231926.22332,3000000000.00000,...,0.02850,772984781.66000,NaN,2024-08-15T08:36:00.000Z,1027.00000,Ethereum,ETH,ethereum,0x3845badAde8e6dFF049820680d1F14bD3903a5d0,2024-08-15 09:37:59.840381
602,7334.00000,Conflux,CFX,conflux-network,172.00000,2020-11-11T00:00:00.000Z,"['mineable', 'pow', 'platform', 'collectibles-...",NaN,4332169169.69000,5032169185.52000,...,0.02840,679818380.04000,NaN,2024-08-15T08:35:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:37:59.840381
603,7080.00000,Gala,GALA,gala,452.00000,2020-09-16T00:00:00.000Z,"['collectibles-nfts', 'gaming', 'binance-smart...",NaN,33973457740.86217,35997617876.87681,...,0.02790,609068678.08000,NaN,2024-08-15T08:35:00.000Z,1027.00000,Ethereum,ETH,ethereum,0xd1d2Eb1B1e90B638588728b4130137D262C87cae,2024-08-15 09:37:59.840381
604,30171.00000,Ethena,ENA,ethena,178.00000,2024-03-29T04:42:08.000Z,"['defi', 'binance-labs-portfolio', 'okx-ventur...",NaN,1806250000.00000,15000000000.00000,...,0.02750,4714951578.90000,NaN,2024-08-15T08:35:00.000Z,1027.00000,Ethereum,ETH,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,2024-08-15 09:37:59.840381


In [39]:
#Fixing the scientific notation on the dataframe by formatting its display

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1.00000,Bitcoin,BTC,bitcoin,11635.00000,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",21000000.00000,19739978.00000,19739978.00000,False,NaN,1.00000,NaN,NaN,NaN,2024-08-15T08:34:00.000Z,58276.62849,32850413658.76016,14.42870,0.55708,-4.29844,1.77572,-6.73944,-12.05270,-11.92583,1150379364264.92725,55.77930,1223809198245.47998,NaN,2024-08-15T08:34:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
1,1027.00000,Ethereum,ETH,ethereum,9211.00000,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,120277040.25230,120277040.25230,True,NaN,2.00000,NaN,NaN,NaN,2024-08-15T08:33:00.000Z,2613.59716,15945122022.59701,5.46140,0.47760,-4.42060,7.86753,-22.22204,-26.50584,-14.17318,314355730647.87018,15.24350,314355730647.87000,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
2,825.00000,Tether USDt,USDT,tether,94195.00000,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'ava...",NaN,116532189829.29184,118060317087.51749,True,NaN,3.00000,NaN,NaN,NaN,2024-08-15T08:33:00.000Z,0.99997,54700349262.82257,11.37720,-0.00340,-0.03989,-0.02836,-0.07465,0.05540,-0.01863,116528577216.14636,5.64760,118056657100.92000,NaN,2024-08-15T08:33:00.000Z,1027.00000,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2024-08-15 09:35:58.183530
3,1839.00000,BNB,BNB,bnb,2209.00000,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,145936326.94494,145936326.94494,False,NaN,4.00000,NaN,NaN,NaN,2024-08-15T08:33:00.000Z,518.02384,1628233232.07296,-4.52090,0.56125,-1.62592,6.18153,-7.02361,-14.46536,-10.08410,75598497131.34264,3.66480,75598497131.34000,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
4,5426.00000,Solana,SOL,solana,716.00000,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,466835260.78769,582259423.78854,True,NaN,5.00000,NaN,NaN,NaN,2024-08-15T08:33:00.000Z,142.42640,2773531902.68569,31.02450,0.68153,-2.23797,-7.71110,-6.96060,-1.37587,-15.76704,66489663385.44262,3.22420,82929110850.32001,NaN,2024-08-15T08:33:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:35:58.183530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,6210.00000,The Sandbox,SAND,the-sandbox,573.00000,2020-08-05T00:00:00.000Z,"['collectibles-nfts', 'content-creation', 'gam...",NaN,2284231926.22332,3000000000.00000,False,NaN,96.00000,NaN,NaN,NaN,2024-08-15T08:36:00.000Z,0.25766,45892456.46669,17.25710,1.32485,-1.30935,3.53600,-20.68918,-32.82563,-41.80801,588558838.91931,0.02850,772984781.66000,NaN,2024-08-15T08:36:00.000Z,1027.00000,Ethereum,ETH,ethereum,0x3845badAde8e6dFF049820680d1F14bD3903a5d0,2024-08-15 09:37:59.840381
602,7334.00000,Conflux,CFX,conflux-network,172.00000,2020-11-11T00:00:00.000Z,"['mineable', 'pow', 'platform', 'collectibles-...",NaN,4332169169.69000,5032169185.52000,True,NaN,97.00000,4332166388.64000,585251852.19262,NaN,2024-08-15T08:35:00.000Z,0.13509,41136962.55539,50.67840,1.08508,-3.13251,9.75976,-18.10374,-26.91030,-38.57964,585252227.89718,0.02840,679818380.04000,NaN,2024-08-15T08:35:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-08-15 09:37:59.840381
603,7080.00000,Gala,GALA,gala,452.00000,2020-09-16T00:00:00.000Z,"[